## Import Package

In [1]:
import numpy as np


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

import os

## Data Import

In [2]:
path = '../../../data/intel_image_classification/'

path_training = path + 'seg_train/seg_train/' # traininng set
path_valid = path + 'seg_test/seg_test/' # vaildation set
path_pred = path + 'seg_pred/seg_pred/' # prediction set

# labels
labels = os.listdir(path_training) # os.listdir(path_valid)

### Import Training/Valid Data + Image Generator

In [3]:
print("Training")
for label in labels:
    print(f'  {label}: {len(os.listdir(path_training+label))}')
    
print()

print("Validation")
for label in labels:
    print(f'  {label}: {len(os.listdir(path_valid+label))}')

Training
  forest: 2271
  mountain: 2512
  street: 2382
  sea: 2274
  glacier: 2404
  buildings: 2191

Validation
  forest: 474
  mountain: 525
  street: 501
  sea: 510
  glacier: 553
  buildings: 437


In [4]:

train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(path_training,
                                                  target_size=(150,150),
                                                  batch_size=20,
                                                  color_mode='rgb')
"""

                                                  width_shift_range=0.2,
                                                  height_shift_range=0.2,
                                                  shear_range=0.2,
                                                  zoom_range=0.2,
                                                  fill_mode='nearest',
                                                  class_mode='cagegorical'
"""


valid_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = valid_datagen.flow_from_directory(path_valid,
                                                  target_size=(150,150))


Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [5]:
from tensorflow.keras.layers import Layer

In [7]:
class IdentityBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='')
        
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.act = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
        
    def call(self, input_tensor):
        X = self.conv1(input_tensor)
        X = self.bn1(X)
        X = self.act(X)
        
        X = self.conv2(X)
        X = self.bn2(X)
        
        X = self.add([X, input_tensor])
        X = self.act(X)
        return X
    
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3))

        # Use the Identity blocks that you just defined
        self.id1a = IdentityBlock(64, 3)
        self.id1b = IdentityBlock(64, 3)

        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        
        self.flatten = tf.keras.layers.Flatten()
        self.drop = tf.keras.layers.Dropout(0.4)
        self.dense_1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(512, activation='relu')
        self.classifier = tf.keras.layers.Dense(num_classes, activation='relu')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)

        # insert the identity blocks in the middle of the network
        x = self.id1a(x)
        x = self.id1b(x)
        x = self.global_pool(x)
        
        x = self.flatten(x)
        x = self.drop(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        
        return self.classifier(x)

In [8]:
resnet = ResNet(len(labels))

In [9]:
resnet.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
               loss=tf.keras.losses.CategoricalCrossentropy(),
               metrics=['accuracy'])

/home/jaewon/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [10]:
history = resnet.fit(train_generator,
           epochs=100,
           validation_data=test_generator)

Epoch 1/100
702/702 [==============================] - 21s 29ms/step - loss: 1.6813 - accuracy: 0.3945 - val_loss: 1.3976 - val_accuracy: 0.4170
Epoch 2/100
702/702 [==============================] - 20s 29ms/step - loss: 1.4731 - accuracy: 0.4039 - val_loss: 1.3509 - val_accuracy: 0.5103
Epoch 3/100
702/702 [==============================] - 20s 29ms/step - loss: 1.3792 - accuracy: 0.4731 - val_loss: 1.1893 - val_accuracy: 0.5727
Epoch 4/100
702/702 [==============================] - 20s 29ms/step - loss: 1.3387 - accuracy: 0.4969 - val_loss: 1.4338 - val_accuracy: 0.3553
Epoch 5/100
702/702 [==============================] - 20s 28ms/step - loss: 1.3504 - accuracy: 0.4520 - val_loss: 1.2099 - val_accuracy: 0.5447
Epoch 6/100
702/702 [==============================] - 20s 29ms/step - loss: 1.2761 - accuracy: 0.5257 - val_loss: 1.2893 - val_accuracy: 0.5357
Epoch 7/100
702/702 [==============================] - 20s 28ms/step - loss: 1.2405 - accuracy: 0.5418 - val_loss: 2.5395 - val_ac

KeyboardInterrupt: 

In [11]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

NameError: name 'history' is not defined

In [12]:
# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [47]:
print("Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])

Accuracy: 0.8950406312942505
Validation Accuracy: 0.82833331823349
